In [ ]:
import csv, json, os, numbers, pandas, re, scipy, scipy.sparse, shutil
import subprocess, sys, threading, time, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [ ]:
pandas.options.display.max_colwidth = 300

## Download ACS2015 File Templates for 5-year and 1-year data

In [ ]:
src = 'https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/2015_1yr_Summary_FileTemplates.zip'
dest = 'capture/ACS2015_1year/2015_1yr_Summary_FileTemplates.zip'
download_file(src, dest)
templates = unzip_file(dest)

src = 'https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/2015_5yr_Summary_FileTemplates.zip'
dest = 'capture/ACS2015_5year/2015_5yr_Summary_FileTemplates.zip'
download_file(src, dest)
templates = unzip_file(dest)

In [ ]:
!ls -l capture/ACS2015_1year/2015_1yr_Summary_FileTemplates/Templates | head

## Download ACS2015 5-year data (tract and block group)

In [ ]:
!wget --header="User-Agent: Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.97 Safari/537.11" --header="Referer: http://xmodulo.com/" https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/5_year_entire_sf/Tracts_Block_Groups_Only.tar.gz

!mkdir -p capture/ACS2005_5year
!mv Tracts_Block_Groups_Only.tar.gz capture/ACS2005_5year

!cd capture/ACS2005_5year; tar xvfz Tracts_Block_Groups_Only.tar.gz >/dev/null

!wget --header="User-Agent: Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.97 Safari/537.11" https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/5_year_entire_sf/2015_ACS_Geography_Files.zip

!mv 2015_ACS_Geography_Files.zip capture/ACS2005_5year

unzip_file('capture/ACS2005_5year/2015_ACS_Geography_Files.zip')

## Read CSV utility functions

In [ ]:
def read_acs2015_5year_template(seqno):
    path = 'capture/ACS2015_5year/2015_5yr_Summary_FileTemplates/2015_5yr_Templates/Seq%d.xls' % seqno
    if not os.path.exists(path):
        return None
    return pandas.read_excel(path)

# Combine template header and data into pandas frame
def read_acs2015_year_data(state, seqno):
    header = read_acs2015_5year_template(seqno)
    data = pandas.read_csv('capture/ACS2015_5year/group2/e20155%s%04d000.txt' % (state, seqno),
                            index_col=False,
                            dtype={'FILEID':numpy.str,
                                   'FILETYPE':numpy.str,
                                   'STUSAB':numpy.str,
                                   'CHARITER':numpy.str,
                                   'SEQUENCE':numpy.str,
                                   'LOGRECNO':numpy.str},
                            header=None,
                            names=header.columns.values)
    return data

In [ ]:
dataset = 'acs2015_5year_tract2010'
column_dir = 'columncache'

## Write ACS2015 5-year description.html

In [ ]:
description_path = column_dir + '/' + dataset + '/description.html'
force_regenerate = True

if os.path.exists(description_path) and not force_regenerate:
    print '{description_path} already exists, skipping'.format(**locals())
else:
    table_rows = []

    for seqno in range(1, 1000):
        template = read_acs2015_5year_template(seqno)
        if template is None:
            break
        for col in range(6, template.shape[1]):
            colname = template.columns.values[col]
            description = template.iloc[0,col]
            description = description.replace(':', '')
            description = re.sub(r'\s*%\s*', ' &mdash; ', description)
            table_rows.append(u'<tr><td>{dataset}.{colname}</td><td>{description}</td></tr>\n'.format(**locals()))

    html = '<table>' + ''.join(table_rows) + '</table>'

    open(description_path, 'w').write(html.encode('utf8'))
    print 'Wrote %d column names and descriptions to %s' % (len(table_rows), description_path)

## Create ACS2015 block-level population

### Read 2010 block geoids and 2010 block populations

In [ ]:
block_populations = numpy.load('columncache/census2010_block2010/p001001.numpy')
print 'block_populations has', sum(block_populations), 'total people'

In [ ]:
# block_geoids_2010 = [row[0] for row in query_psql("SELECT geoid2010 FROM sf1_2010_block_p001 order by blockidx2010")]
block_geoids_2010 = json.load(open('block_geoids_2010.json'))
print 'There are', len(block_geoids_2010), 'blocks'

assert(len(block_geoids_2010) + 1 == len(block_populations))

### Compute 2010 population by tract and block indices from tract


In [ ]:
tract_populations = {}
tract_block_indexes = {}

for block_index_minus_one, block_geoid in enumerate(block_geoids_2010):
    block_index = block_index_minus_one + 1
    tract_name = block_geoid[0:11]
    if tract_name not in tract_populations:
        tract_populations[tract_name] = 0
        tract_block_indexes[tract_name] = []
    tract_populations[tract_name] += block_populations[block_index]
    tract_block_indexes[tract_name].append(block_index)

print 'There are', len(tract_populations), 'tracts'
print 'tract_populations has', sum(tract_populations.values()), 'people'

### Map tract identifiers to LOGRECNO using geography file

In [ ]:
tract_to_logrecno = {}

def compute_tract_to_logrecno(state):
    geography = pandas.read_csv('capture/ACS2015_5year/2015_ACS_Geography_Files/g20155{state}.csv'.format(**locals()),
                                dtype=numpy.str,
                                index_col=False,
                                header=None,
                                keep_default_na=False,
                                na_values=[])

    nrows = geography.shape[0]
    print 'State {state} has {nrows} geography rows'.format(**locals())
    
    ntracts = 0
    tract_to_logrecno[state] = {}
    
    for r in range(0, geography.shape[0]):
        aggregation_level = geography.iloc[r, 2]
        if aggregation_level == '140': # census tract
            tract_identifier = geography.iloc[r, 48][7:]
            logrecno = geography.iloc[r, 4]
            tract_to_logrecno[state][tract_identifier] = logrecno
    
    print 'Found %d tracts for state %s' % (len(tract_to_logrecno[state]), state)

for state in state_names:
    compute_tract_to_logrecno(state)

### Interpolate and write columns for data file

In [ ]:
# TODO: can we do this with a data frame then write out columns?

def interpolate_acs_file(state, seq):
    print 'Reading %s:%d' % (state, seq)
    data = read_acs2015_year_data(state, seq)

    print 'Mapping locrecno to row'
    logrecnos = data['LOGRECNO']

    logrecno_to_row = {}

    for r, logrecno in enumerate(logrecnos):
        logrecno_to_row[logrecno] = r
    
    col_names = data.columns.values[6:]
    print 'Iterating across %d columns' % len(col_names)
    for col_name in col_names:
        input_col = data[col_name]
        output_col_path = column_dir + '/' + dataset + '/' + col_name + '.float32'
        if os.path.exists(output_col_path):
            print '%s already exists, skipping' % output_col_path
            continue

        output_col = numpy.zeros(block_populations.size, dtype=numpy.float32)

        for tract in sorted(tract_to_logrecno[state].keys()):
            input_pop = input_col[logrecno_to_row[tract_to_logrecno[state][tract]]]
            if not isinstance(input_pop, numbers.Number):
                if input_pop == '.':
                    input_pop = 0
                else:
                    try:
                        input_pop = float(input_pop)
                    except:
                        print 'That population is'
                        print input_pop
                        print type(input_pop)
                        print '>%s<' % input_pop
                        input_pop = 0
            if not tract in tract_block_indexes:
                print 'missing tract {tract} from tract_block_indexes'.format(**locals())
            else:
                for block_index in tract_block_indexes[tract]:
                    if block_populations[block_index]:
                        output_col[block_index] = input_pop * float(block_populations[block_index]) / tract_populations[tract]
            
        output_col.tofile(output_col_path + '.tmp')
        os.rename(output_col_path + '.tmp', output_col_path)
        print 'Created %s' % output_col_path

for seq in range(97, 2000):
    interpolate_acs_file('pa', seq)

In [ ]:
# TODO: can we do this with a data frame then write out columns?

def interpolate_acs_file(seq):
    output_cols = {}
    missing_tracts = {}
    for state in state_names:
        data = read_acs2015_year_data(state, seq)
    
        logrecnos = data['LOGRECNO']

        logrecno_to_row = {}

        col_names = data.columns.values[6:]
        print '%s:%d has %d columns' % (state, seq, len(col_names))
        assert len(col_names) < 500   # sanity check to avoid demanding too much RAM on hal15

        for r, logrecno in enumerate(logrecnos):
            logrecno_to_row[logrecno] = r
    
        for col_name in col_names:
            input_col = data[col_name]
                
            if not col_name in output_cols:
                output_cols[col_name] = numpy.zeros(block_populations.size, dtype=numpy.float32)
            output_col = output_cols[col_name]

            for tract in sorted(tract_to_logrecno[state].keys()):
                input_pop = input_col[logrecno_to_row[tract_to_logrecno[state][tract]]]
                if not isinstance(input_pop, numbers.Number):
                    if input_pop == '.':
                        input_pop = 0
                    else:
                        try:
                            input_pop = float(input_pop)
                        except:
                            print 'That population is'
                            print input_pop
                            print type(input_pop)
                            print '>%s<' % input_pop
                            input_pop = 0
                            
                            
                if not tract in tract_block_indexes:
                    missing_tracts[tract] = True
                else:
                    for block_index in tract_block_indexes[tract]:
                        if block_populations[block_index]:
                            output_col[block_index] = input_pop * float(block_populations[block_index]) / tract_populations[tract]
            
    print 'Missing tracts: %s' % (sorted(missing_tracts.keys()))

    for col_name in sorted(output_cols.keys()):
        output_col_path = column_dir + '/' + dataset + '/' + col_name + '.float32'
        output_cols[col_name].tofile(output_col_path + '.tmp')
        os.rename(output_col_path + '.tmp', output_col_path)
        print 'Created %s with sum %f' % (output_col_path, output_cols[col_name].sum())
    
        
for seq in range(1, 1000):
    interpolate_acs_file(seq)

In [ ]:
len(tract_block_indexes.keys())

In [ ]:
!ls -l columncache/acs2015_5year_tract2010/B08006_002.float32

In [ ]:
x=numpy.memmap('columncache/acs2015_5year_tract2010/B08006_002.float32', dtype=numpy.float32, mode='r')

In [ ]:
x.sum()